In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [ ]:
## Lets get dataset and perform some pre processing

In [ ]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd

In [ ]:
dataset1 = load_dataset('marmikpandya/mental-health')
dataset2 = load_dataset("nbertagnolli/counsel-chat")
dataset3 = load_dataset("Amod/mental_health_counseling_conversations")

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
dataset1 = dataset1['train'].to_pandas()
dataset2 = dataset2['train'].to_pandas()
dataset3 = dataset3['train'].to_pandas()

dataset1.dropna(inplace = True)
dataset2.dropna(inplace = True)
dataset3.dropna(inplace = True)

In [ ]:
dataset1.shape ,dataset2.shape ,dataset3.shape

((13358, 3), (2612, 10), (3512, 2))

In [ ]:
dataset1['text_len'] = dataset1['output'].apply(len)
dataset2['text_len'] = dataset2['answerText'].apply(len)
dataset3['text_len'] = dataset3['Response'].apply(len)

In [ ]:
def print_binvalues(df) :
  bin_edges = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, np.inf]
  bins = pd.cut(df['text_len'], bins=bin_edges, right=False)
  count_by_interval = bins.value_counts().sort_index()
  print("Count of rows by text_length intervals:")
  print(count_by_interval)

In [ ]:
print_binvalues(dataset1)
print_binvalues(dataset2)
print_binvalues(dataset3)

In [ ]:
dataset1 = dataset1.rename(columns={'input': 'questionText', 'output': 'answerText'})
dataset3 = dataset3.rename(columns={'Context': 'questionText', 'Response': 'answerText'})

In [ ]:
# filter to get rows which has length of 1000 or less
dataset1 = dataset1[dataset1['text_len'] <= 1000]
dataset2 = dataset2[dataset2['text_len'] <= 1000]
dataset3 = dataset3[dataset3['text_len'] <= 1000]

In [ ]:
# remove every other column
dataset1 = dataset1[['questionText','answerText']]
dataset2 = dataset2[['questionText','answerText']]
dataset3 = dataset3[['questionText','answerText']]

In [ ]:
dataset1.shape ,dataset2.shape ,dataset3.shape

((11988, 2), (1682, 2), (2142, 2))

In [ ]:
# concat the datasets
df = pd.concat([dataset1, dataset2,dataset3], ignore_index=True)

In [ ]:
df.shape

(15812, 2)

In [ ]:
## convert to hugging face dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['questionText', 'answerText'],
    num_rows: 15812
})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

#set the qunatization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
#
#Load the model and Tokenizer
model_id = "microsoft/phi-2"
#
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True,use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 17.06 MiB is free. Process 21640 has 14.72 GiB memory in use. Of the allocated memory 14.08 GiB is allocated by PyTorch, and 516.90 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
SYSTEM_PROMPT2 = '''You are a compassionate and truthful assistant for psychological and emotional support. Provide empathetic, non-judgmental responses, actively listening and prioritizing the user's feelings and safety. If users express self-harm or harming others, encourage seeking immediate professional help. Avoid diagnosing or prescribing treatments, instead guiding users to consult licensed therapists or medical professionals. Remain impartial, creating a safe space for users to share without personal opinions. Your goal is a supportive, understanding environment focused on the user's well-being.'''

def format_llama(entry):
  formatted = f"<s>[INST] <<SYS>>{SYSTEM_PROMPT2}<</SYS>>{entry['questionText']} [/INST]  {entry['answerText']}  </s>"

  return formatted

# add the "prompt" column in the dataset
text_column = [format_llama(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [ ]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [ ]:
train_data = dataset

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)
#
modules = find_all_linear_names(model)
print(modules)

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

# Enable gradient checkpointing for memory efficiency during training
model.gradient_checkpointing_enable()

# Prepare the model for training with k-bit precision (if applicable)
model = prepare_model_for_kbit_training(model)

# Print the model architecture
print(model)

# Define Lora configuration
lora_config = LoraConfig(
    r=32, # A lower rank value for r can significantly reduce the memory footprint and computational requirements
    lora_alpha=16,
    target_modules=modules,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

In [ ]:
train = dataset.select(range(14000))
val = dataset.select(range(14000,15000))

In [ ]:
import transformers
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side='right'
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=val,
    dataset_text_field="prompt",
    peft_config=lora_config,
    max_seq_length=2000,
    args=transformers.TrainingArguments(
        output_dir="TherapistPhi2",
        logging_dir='./results',
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=70,
        lr_scheduler_type="constant",
        learning_rate=2e-4,
        logging_steps=1,
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
#
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
new_model = "phi-therapist-chat-v1"

trainer.model.save_pretrained(new_model)

base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
#save_adapter=True, save_config=True
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
merged_model.push_to_hub("HarleyQuin/phi-therapist-chat-v1", use_temp_dir=False)
tokenizer.push_to_hub("HarleyQuin/phi-therapist-chat-v1", use_temp_dir=False)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

! zip -r merged_model.zip merged_model

In [ ]:
df['questionText'][0]

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"
  prompt = f"<s>[INST] <<SYS>>{SYSTEM_PROMPT2}<</SYS>>{query} [/INST]"

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)
#
query = """I'm feeling really anxious lately and I don't know why."""

result = get_completion(query=query, model=merged_model, tokenizer=tokenizer)
print(result)